# Models

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lukeconibear/intro_ml/blob/main/docs/03_models.ipynb)

In [1]:
# if you're using colab, then install the required modules
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    pass

## Hyperparameter tuning

Choosing the best model

`keras_tuner`

In [ ]:
import keras_tuner

https://keras.io/keras_tuner/

...

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

2022-03-14 16:25:28.299470: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-14 16:25:28.299487: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
if tf.config.list_physical_devices("GPU"):
    print(
        f"Yes, there are {len(tf.config.list_physical_devices('GPU'))} GPUs available."
    )
else:
    print("No, GPUs are not available.")

No, GPUs are not available.


2022-03-14 16:25:29.209000: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-14 16:25:29.209027: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-14 16:25:29.209044: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (UOL-LAP-5G6CZH3): /proc/driver/nvidia/version does not exist


[Keras Applications](https://keras.io/api/applications/)

- DenseNet
- VGG
- EfficientNet
- MobileNet
- Inception
- ResNet

In [15]:
tf.keras.applications.inception_v3.InceptionV3?

Signature:
tf.keras.applications.inception_v3.InceptionV3(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax',
)
Docstring:
Instantiates the Inception v3 architecture.

Reference:
- [Rethinking the Inception Architecture for Computer Vision](
    http://arxiv.org/abs/1512.00567) (CVPR 2016)

This function returns a Keras image classification model,
optionally loaded with weights pre-trained on ImageNet.

For image classification use cases, see
[this page for detailed examples](
  https://keras.io/api/applications/#usage-examples-for-image-classification-models).

For transfer learning use cases, make sure to read the
[guide to transfer learning & fine-tuning](
  https://keras.io/guides/transfer_learning/).

Note: each Keras Application expects a specific kind of input preprocessing.
For `InceptionV3`, call `tf.keras.applications.inception_v3.preprocess_input`
on your inputs b

https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/

In [4]:
conv_base = tf.keras.applications.vgg16.VGG16(
    weights="imagenet", include_top=False, input_shape=(180, 180, 3)
)

2022-03-14 16:25:29.237337: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


58900480/58889256 [==============================] - 5s 0us/step


In [5]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 180, 180, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 180, 180, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 90, 90, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 90, 90, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 90, 90, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 45, 45, 128)       0     

## Pretrained

...

example

In [1]:
import tensorflow as tf

2022-03-17 16:27:12.602805: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-17 16:27:12.602823: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
# # Load a convolutional base with pre-trained weights
# base_model = tf.keras.applications.Xception(
#     weights='imagenet',
#     include_top=False,
#     pooling='avg')

# # Freeze the base model
# base_model.trainable = False

# Use a Sequential model to add a trainable classifier on top
# model = tf.keras.Sequential(
#     [
#         base_model,
#         tf.keras.layers.Dense(1000),
#     ]
# )

# # Compile & train
# model.compile(...)
# model.fit(...)

In [6]:
# model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 2048)              20861480  
                                                                 
 dense (Dense)               (None, 1000)              2049000   
                                                                 
Total params: 22,910,480
Trainable params: 2,049,000
Non-trainable params: 20,861,480
_________________________________________________________________


## [Callbacks](https://keras.io/api/callbacks/)

Callbacks are objects that get called by the model at different points during training, in particular after each batch or epoch.

For example, they could be used to:

- Save a model version at regularly intervals or once attained a metric threshold (i.e., checkpointing).
- Monitor and profile the training progress (i.e., TensorBoard)
- Change the learning rate when training plateaus.
- Fine tuning when the training plateaus.

### [Checkpointing](https://www.tensorflow.org/guide/checkpoint)

For longer or distributed training, it's helpful to save the model (/ model weights) at regular intervals in case it crashes during training (i.e. [model checkpointing](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint)).

In [9]:
callback_model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="models/model_{epoch}",
    save_freq="epoch",  # save a model version at the end of each epoch
    save_best_only=True,  # only save a model if val_accuracy improved
    monitor="val_accuracy",
)

### Tensorboard and Profiling

[TensorBoard](https://www.tensorflow.org/tensorboard) is a browser-based application that provides live plots of loss and metrics for training and evaluation.

You can add [profiling](https://www.tensorflow.org/guide/profiler) too.

In [10]:
callback_tensorboard_with_profiling = tf.keras.callbacks.TensorBoard(
    log_dir="logs",
    profile_batch=(1, 5),  # profile batches 1 to 5
    update_freq="epoch",
)

2022-03-17 18:23:38.197379: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-03-17 18:23:38.197406: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-03-17 18:23:38.197650: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.


View them with:

In [ ]:
# !tensorboard --logdir=/full_path_to_your_logs

Also, in-line in [Jupyter Notebooks / Google Colab](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks)

### Metric threshold

Stop training when a monitored metric has stopped improving (i.e., [early stopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping)).

In [8]:
callback_accuracy_threshold = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",  # quantity to be monitored
    min_delta=1e-2,  # "no longer improving" means "stopped improving by at least 1e-2"
    patience=2,  # "no longer improving" also means "for at least 2 epochs"
    verbose=1,
)

### Learning rate decay

Reduce learning rate when a metric has stopped improving (i.e., [reduce the learning rate on plateau](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau)).

In [ ]:
callback_learning_rate_decay = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_accuracy",
    patience=5,
    factor=0.2,  # factor by which the learning rate will be reduced: new_lr = lr * factor
    min_lr=0.001,  # lower bound on the learning rate
)

### Example: Callbacks

In [ ]:
callbacks = [
    callback_model_checkpoint,
    callback_tensorboard_with_profiling,
    callback_accuracy_threshold,
    callback_learning_rate_decay,
]

In [ ]:
# model.fit(dataset, epochs=5, callbacks=callbacks)

## Compiling

Compile any function in TensorFlow by wrapping it in the [`@tf.function`](https://www.tensorflow.org/api_docs/python/tf/function) decorator.

This convert it from eager execution to a static graph.



## Exercises

```{admonition} Exercise 1

...

```

## {ref}`Solutions <models>`

## Key Points

```{important}

- [x] _..._

```

## Further information

### Good practices

- See if there is a model architecture (and parameters) that already addresses the task.


### Other options

- ...
 
### Resources

#### General

- [Model Zoo](https://modelzoo.co/)
- [Papers with code - Models](https://paperswithcode.com/methods)
- [HuggingFace - Models](https://huggingface.co/models)

#### TensorFlow

- [TensorFlow Hub](https://tfhub.dev/) for pre-trained models.
- [TensorFlow Model Garden](https://github.com/tensorflow/models/tree/master/official) for model source code.

#### PyTorch

- [PyTorch Hub](https://pytorch.org/docs/stable/hub.html) for pre-retrained models.
- [Torch Vision Models](https://pytorch.org/vision/stable/models.html)
- [Torch Text Models](https://pytorch.org/text/stable/models.html)
- [Torch Audio Models](https://pytorch.org/audio/stable/models.html)
- [TIMM (pyTorch IMage Models)](https://rwightman.github.io/pytorch-image-models/)